# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.
  - **Director**: The director of the movie.
  - **Stars**: The main stars of the movie.
  - **Votes**: The number of votes the movie received.
  - **Gross ($M)**: The gross earnings of the movie in millions of USD.

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

In [5]:
# Your solution goes here

import requests
from bs4 import BeautifulSoup
import pandas as pd
import math

def build_url(title_type, user_rating, start_date, end_date):
    base_url = "https://www.imdb.com/search/title/"
    url = f"{base_url}?title_type={title_type}&user_rating={user_rating},&release_date={start_date},{end_date}&count=50"
    return url

def scrape_imdb(title_type, user_rating, start_date, end_date):
    url = build_url(title_type, user_rating, start_date, end_date)
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Failed to retrieve data: {response.status_code}")
        return pd.DataFrame()

    soup = BeautifulSoup(response.content, 'html.parser')
    movies_list = soup.find_all('div', class_='lister-item mode-advanced')

    data = []
    
    for idx, movie in enumerate(movies_list):
        # Extract title
        title = movie.h3.a.text
        
        # Extract release year
        year = movie.h3.find('span', class_='lister-item-year').text.strip('()')

        # Extract IMDb rating
        rating = movie.find('div', class_='inline-block ratings-imdb-rating')
        rating = rating['data-value'] if rating else None

        # Extract runtime
        runtime = movie.find('span', class_='runtime')
        runtime = runtime.text.split()[0] if runtime else None
        
        # Extract genre
        genre = movie.find('span', class_='genre')
        genre = genre.text.strip() if genre else None

        # Extract description
        description = movie.find_all('p', class_='text-muted')[1].text.strip()

        # Extract director and stars
        credit = movie.find('p', class_='')
        directors = []
        stars = []
        for a in credit.find_all('a'):
            if 'adv_li_dr' in a['href']:
                directors.append(a.text)
            if 'adv_li_st' in a['href']:
                stars.append(a.text)

        # Extract votes and gross earnings
        votes_and_gross = movie.find_all('span', attrs={'name': 'nv'})
        votes = votes_and_gross[0].text if len(votes_and_gross) > 0 else None
        gross = votes_and_gross[1].text if len(votes_and_gross) > 1 else None

        # Append to data list
        data.append({
            'Movie Nr': idx + 1,
            'Title': title,
            'Year': year,
            'Rating': rating,
            'Runtime (min)': runtime,
            'Genre': genre,
            'Description': description,
            'Director': ', '.join(directors),
            'Stars': ', '.join(stars),
            'Votes': votes,
            'Gross ($M)': gross
        })

    # Convert to DataFrame
    df = pd.DataFrame(data)
    return df

In [6]:
# Example usage
df = scrape_imdb('feature', '7.5', '1990-01-01', '1992-12-31')
print(df.head())

Failed to retrieve data: 403
Empty DataFrame
Columns: []
Index: []


## BONUS

The search results span multiple pages, housing a total of 631 movies in our example with each page displaying 50 movies at most. To scrape data seamlessly from all pages, you'll need to dive deep into the structure of the URLs generated with each "Next" click.

Take a close look at the following URLs:
- First page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,
  ```
- Second page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt
  ```
- Third page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt
  ```

You should notice a pattern. There is a `start` parameter incrementing by 50 with each page, paired with a constant `ref_` parameter holding the value "adv_nxt".

Modify your script so it's capable of iterating over all available pages to fetch data on all the 631 movies (631 is the total number of movies in the proposed example).

In [7]:
# Your solution goes here
def build_url(title_type, user_rating, start_date, end_date, start=1):
    """
    Construct the URL for IMDb search with pagination.
    """
    base_url = "https://www.imdb.com/search/title/"
    url = f"{base_url}?title_type={title_type}&user_rating={user_rating},&release_date={start_date},{end_date}&start={start}&ref_=adv_nxt"
    return url

def get_total_movies(soup):
    """
    Extract the total number of movies from the search results page.
    """
    results_text = soup.find('div', class_='desc').span.text
    total_movies = int(results_text.split()[-2].replace(',', ''))
    return total_movies

def scrape_page(url):
    """
    Scrape the data from a single page of IMDb search results.
    """
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Failed to retrieve data: {response.status_code}")
        return []
    
    soup = BeautifulSoup(response.content, 'html.parser')
    movies_list = soup.find_all('div', class_='lister-item mode-advanced')

    data = []
    
    for idx, movie in enumerate(movies_list):
        # Extract title
        title = movie.h3.a.text
        
        # Extract release year
        year = movie.h3.find('span', class_='lister-item-year').text.strip('()')

        # Extract IMDb rating
        rating = movie.find('div', class_='inline-block ratings-imdb-rating')
        rating = rating['data-value'] if rating else None

        # Extract runtime
        runtime = movie.find('span', class_='runtime')
        runtime = runtime.text.split()[0] if runtime else None
        
        # Extract genre
        genre = movie.find('span', class_='genre')
        genre = genre.text.strip() if genre else None

        # Extract description
        description = movie.find_all('p', class_='text-muted')[1].text.strip()

        # Extract director and stars
        credit = movie.find('p', class_='')
        directors = []
        stars = []
        for a in credit.find_all('a'):
            if 'adv_li_dr' in a['href']:
                directors.append(a.text)
            if 'adv_li_st' in a['href']:
                stars.append(a.text)

        # Extract votes and gross earnings
        votes_and_gross = movie.find_all('span', attrs={'name': 'nv'})
        votes = votes_and_gross[0].text if len(votes_and_gross) > 0 else None
        gross = votes_and_gross[1].text if len(votes_and_gross) > 1 else None

        # Append to data list
        data.append({
            'Title': title,
            'Year': year,
            'Rating': rating,
            'Runtime (min)': runtime,
            'Genre': genre,
            'Description': description,
            'Director': ', '.join(directors),
            'Stars': ', '.join(stars),
            'Votes': votes,
            'Gross ($M)': gross
        })

    return data

def scrape_imdb(title_type, user_rating, start_date, end_date):
    """
    Scrape IMDb for movies based on title type, user rating, and date range across multiple pages.
    """
    url = build_url(title_type, user_rating, start_date, end_date)
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Failed to retrieve data: {response.status_code}")
        return pd.DataFrame()

    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract total number of movies
    total_movies = get_total_movies(soup)
    print(f"Total number of movies to scrape: {total_movies}")
    
    # Calculate the number of pages
    movies_per_page = 50
    total_pages = math.ceil(total_movies / movies_per_page)

    all_data = []
    
    # Scrape each page
    for page in range(1, total_pages + 1):
        start = (page - 1) * movies_per_page + 1
        url = build_url(title_type, user_rating, start_date, end_date, start=start)
        print(f"Scraping page {page} with URL: {url}")
        
        page_data = scrape_page(url)
        all_data.extend(page_data)

    # Convert to DataFrame
    df = pd.DataFrame(all_data)
    return df



In [8]:
# Example usage
df = scrape_imdb('feature', '7.5', '1990-01-01', '1992-12-31')
print(df.head())

Failed to retrieve data: 403
Empty DataFrame
Columns: []
Index: []
